In [16]:
import pandas as pd
import matrix

In [17]:
requirements_stop_words = ['условия', 'преимущества работы', 'у нас есть', 'предлагаем', 'гарантируем', 'нужно будет']
requirements_start_words = ['Требования', 'Твой функционал', 'мы ожидаем', 'хотим от', 'вас', 'тебя', 'нужно знать',
                            'нужно уметь', 'Ключевые умения', 'ожидания']

def hh_vacancy_requirements_extractor(row):
    for start_word in requirements_start_words:
        for stop_word in requirements_stop_words:
            start_position = row.find(start_word)
            stop_position = row.find(stop_word)
            if stop_position > start_position:
                return row[start_position + len(start_word): stop_position]
    return ''

skills_mark_words = [('знать', '1 4 объем дисциплины'),
                     ('содержание дисциплины', '3 распределение учебного времени')]


def extract(doc, mark_sent):
    mark_start, mark_end = mark_sent
    start_position = doc.find(mark_start)
    end_position = doc.find(mark_end)
    if start_position == -1 or end_position == -1 or start_position > end_position:
        return ''
    return doc[start_position + len(mark_start): end_position] + extract(doc[end_position + len(mark_end):], mark_sent)


def education_skills_extractor(document):
    result = extract(document, skills_mark_words[0])
    content = extract(document, skills_mark_words[1])
    return ' '.join([result, content])


In [3]:
#Tf-Idf для документов hh
data_set = pd.read_csv('./merged.csv')
documents = data_set['description']
term, documents_matrix = matrix.DocumentMatrixBuilder().build_matrix(documents, hh_vacancy_requirements_extractor, min_df=0.001)
matrix.save_matrix_and_terms(term, documents_matrix)

In [11]:
#Tf-Idf для документов обр программ по модулям
data_set = pd.read_csv('./ed.csv')
name = data_set['name']
documents = data_set['documents']

term, documents_matrix = matrix.DocumentMatrixBuilder().build_matrix(
    documents, education_skills_extractor, min_df=0.005)
matrix.save_matrix_and_terms(term, documents_matrix, './ed_matrix')

In [18]:
#Tf-Idf для документов обр программ по дисцплинам
data_set = pd.read_csv('./ed_by_dist.csv')
name = data_set['name']
documents = data_set['documents']

term, documents_matrix = matrix.DocumentMatrixBuilder().build_matrix(
    documents, education_skills_extractor, min_df=0.005)
matrix.save_matrix_and_terms(term, documents_matrix, './ed_matrix_dist')

In [19]:
def merge_vocabulary(_in, _this):
    counter = len(_in)
    for key in _this.keys():
        if key in _in:
            continue
        _in[key] = counter
        counter = counter + 1
    return _in

def save_vocabulary(voc, file_name):
    with open(file_name, 'w', encoding='utf-8') as fd:
        for word in voc:
            fd.write('{}\n'.format(word))

def get_corpus_and_vocabulary(data, competency_extractor, min_df=0.05, max_df=0.99):
    document_matrix_builder = matrix.DocumentMatrixBuilder()
    corpus, vocabulary = document_matrix_builder.get_prepocessing_text_and_vocabulary(data, min_df=min_df,
                                                                                        max_df=max_df,
                                                                                        sub_text_extractor=competency_extractor)
    return corpus, vocabulary 

In [14]:
# общий Tf-Idf для документов обр программ по модулям и hh
data_set = pd.read_csv('./merged.csv')
hh_vacancy = data_set['description']
data_set = pd.read_csv('./ed.csv')
educations = data_set['documents']

hh_corpus, hh_vocabulary = get_corpus_and_vocabulary(hh_vacancy,hh_vacancy_requirements_extractor, min_df=0.005, max_df=0.99)
ed_corpus, ed_vocabulary = get_corpus_and_vocabulary(educations, education_skills_extractor, min_df=0.005,max_df=0.99)
save_vocabulary(ed_vocabulary, './ed_voc')


merged_vocabulary = merge_vocabulary(hh_vocabulary, ed_vocabulary)

hh_vector, hh_terms = matrix.DocumentMatrixBuilder().build_matrix_from_corpus_and_vocabulary(hh_corpus, merged_vocabulary)
matrix.save_matrix_and_terms(hh_terms, hh_vector, './com_matrix')

ed_vecor, ed_terms = matrix.DocumentMatrixBuilder().build_matrix_from_corpus_and_vocabulary(ed_corpus, merged_vocabulary)
matrix.save_matrix_and_terms(ed_terms, ed_vecor, './com_ed')

In [20]:
# общий Tf-Idf для документов обр программ по дисциплинам и hh
data_set = pd.read_csv('./merged.csv')
hh_vacancy = data_set['description']
data_set = pd.read_csv('./ed_by_dist.csv')
educations = data_set['documents']

hh_corpus, hh_vocabulary = get_corpus_and_vocabulary(hh_vacancy,hh_vacancy_requirements_extractor, min_df=0.05, max_df=0.99)
ed_corpus, ed_vocabulary = get_corpus_and_vocabulary(educations,education_skills_extractor, min_df=0.005,max_df=0.99)
save_vocabulary(ed_vocabulary, './ed_dist_voc')

merged_vocabulary = merge_vocabulary(hh_vocabulary, ed_vocabulary)

document_matrix_builder = matrix.DocumentMatrixBuilder()
hh_vector, hh_terms = document_matrix_builder.build_matrix_from_corpus_and_vocabulary(hh_corpus, merged_vocabulary)
matrix.save_matrix_and_terms(hh_terms, hh_vector, './com_matrix_dist')

ed_vecor, ed_terms = document_matrix_builder.build_matrix_from_corpus_and_vocabulary(ed_corpus, merged_vocabulary)
matrix.save_matrix_and_terms(ed_terms, ed_vecor, './com_ed_dist')